### Analysing previous trajectory with DRIVER

In [1]:
import plumed

In [2]:
with open('plumed.dat', 'w') as f:
    f.write('''
    # Activate MOLINFO functionalities
    MOLINFO STRUCTURE=solvated.pdb

    # Get the distance between the first and second atom
    d1: DISTANCE ATOMS=1,2

    # Print the distance every step
    PRINT ARG=d1 FILE=COLVAR STRIDE=1
    ''')

In [12]:
dt = 0.004 # in picoseconds
dt_log = 10000 * dt

!plumed driver --plumed plumed.dat --mf_dcd traj.out --timestep {dt_log}


DRIVER: Found molfile format trajectory dcd with name traj.out
dcdplugin) detected standard 32-bit DCD file of native endianness
dcdplugin) CHARMM format DCD file (also NAMD 2.1 and later)
PLUMED: PLUMED is starting
PLUMED: Version: 2.9.2 (git: Unknown) compiled on Sep  4 2024 at 12:28:06
PLUMED: Please cite these papers when using PLUMED [1][2]
PLUMED: For further information see the PLUMED web page at http://www.plumed.org
PLUMED: Root: /opt/homebrew/anaconda3/envs/openmm/lib/plumed
PLUMED: For installed feature, see /opt/homebrew/anaconda3/envs/openmm/lib/plumed/src/config/config.txt
PLUMED: Molecular dynamics engine: driver
PLUMED: Precision of reals: 8
PLUMED: Running over 1 node
PLUMED: Number of threads: 1
PLUMED: Cache line size: 512
PLUMED: Number of atoms: 28504
PLUMED: File suffix: 
PLUMED: FILE: plumed.dat
PLUMED: Action MOLINFO
PLUMED:   with label @0
PLUMED:   pdb file named solvated.pdb contains 4 chains 
PLUMED:   chain named A contains residues 1 to 100 and atoms 1 to

In [19]:
from openmm import *
from openmm.app import *
from openmm.unit import *
from openmm import unit

pdf = PDBFile('solvated.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# System Configuration
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers  #This cutoff and the following for tolerance are standard values
ewaldErrorTolerance = 10**(-4)
constraints = HBonds
rigidWater = True
constraintTolerance = 10**(-4) #Quite a default value for accuracy

# Integration Options
fs = 0.001 * picoseconds
dt = 4 * fs
temperature = 300*kelvin
friction = 1.0/picosecond

# Simulation Options
steps = 10**6
equilibrationSteps = 10**5
platform = Platform.getPlatformByName('OpenCL')
dcdReporter = DCDReporter('traj.out', 10000)
dataReporter = StateDataReporter('logger',10000, totalSteps=steps,
    step=True, time=True, speed=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint', 10000)

# Prepare the Simulation
print('Building system...')
topology = pdf.topology
positions = pdf.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance)


integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator, platform)
simulation.context.setPositions(positions)
# Optionally, add a barostat to the system
barostat = MonteCarloBarostat(1 * unit.atmosphere, 300 * unit.kelvin)
system.addForce(barostat)

## Minimize and Equilibrate. First, minimize ONLY positions of H atoms
## Create a dictionary to map atom indices to their types
atom_types = {}
for atom in topology.atoms():
    atom_types[atom.index] = atom.element.symbol  # Use element symbol or type
#
## Identify hydrogen atoms
hydrogen_indices = [index for index, element in atom_types.items() if element == 'H']
#
## Apply constraints to non-hydrogens
#for i in range(system.getNumParticles()):
#    if i not in hydrogen_indices:
#        # Apply constraints to keep non-hydrogens fixed
#        system.addConstraint(i, i, 0.0)  # Constraint with zero distance

# Add a custom external force to fix atom positions
fix_force = openmm.CustomExternalForce('0.5*k*(x^2 + y^2 + z^2)')
fix_force.addPerParticleParameter('k')
k = 1e10 * unit.kilojoules_per_mole / unit.nanometers**2  # Large force constant
for i in range(system.getNumParticles()):
    if i not in hydrogen_indices:
#for atomIndex in non_hydrogen_atom_indices:
#      fix_force.addParticle(atomIndex, [k])
      fix_force.addParticle(i, [k])
system.addForce(fix_force)


print('Performing energy minimization...')
simulation.minimizeEnergy()

minimized_positions = simulation.context.getState(getPositions=True).getPositions()

#Now recreate the system but with no constraints
system = forcefield.createSystem(topology,
                                      nonbondedMethod=nonbondedMethod,
                                      nonbondedCutoff=nonbondedCutoff,
                                      constraints=HBonds)

print('Initiate second minimization...')
#Set the positions to the values obtained after minimization
simulation.context.setPositions(minimized_positions)

#Re-minimize but allow all atoms to move
simulation.minimizeEnergy()

#Equilibrate
print('Equilibrating...')
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

# Simulate

print('Simulating...')
simulation.reporters.append(dcdReporter)
simulation.reporters.append(dataReporter)
simulation.reporters.append(checkpointReporter)
simulation.currentStep = 0
simulation.step(steps)


Building system...
Performing energy minimization...


KeyboardInterrupt: 